In [1]:
import pandas as pd

In [2]:
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("my-first-experiment")

<Experiment: artifact_location='/workspaces/mlops-camp/notebook/03-experiment-tracking/mlruns/1', creation_time=1716229855130, experiment_id='1', last_update_time=1716229855130, lifecycle_stage='active', name='my-first-experiment', tags={}>

In [4]:
def read_data(file):
    df =pd.read_parquet(file)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td:td.total_seconds()/60)
    df = df[(df.duration >=1) & (df.duration <=60)]

    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df
    

In [5]:
df = pd.read_parquet("./data/green_tripdata_2023-01.parquet")

In [6]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.90,1.0,0.5,4.03,0.0,None,1.0,24.18,1.0,1.0,2.75
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.70,1.0,0.5,2.64,0.0,None,1.0,15.84,1.0,1.0,0.00
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.20,1.0,0.5,1.94,0.0,None,1.0,11.64,1.0,1.0,0.00
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.50,0.5,1.5,1.70,0.0,None,1.0,10.20,1.0,1.0,0.00
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.00,0.5,1.5,0.00,0.0,None,1.0,8.00,1.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68206,2,2023-01-31 22:29:00,2023-01-31 22:42:00,None,NaN,49,62,NaN,4070.82,15.70,0.0,0.0,0.00,0.0,None,1.0,16.70,NaN,NaN,NaN
68207,2,2023-01-31 22:40:00,2023-01-31 22:48:00,None,NaN,10,205,NaN,2.14,4.41,0.0,0.0,0.00,0.0,None,1.0,5.41,NaN,NaN,NaN
68208,2,2023-01-31 23:46:00,2023-02-01 00:02:00,None,NaN,66,37,NaN,3.44,16.53,0.0,0.0,3.51,0.0,None,1.0,21.04,NaN,NaN,NaN
68209,2,2023-01-31 23:01:00,2023-01-31 23:19:00,None,NaN,225,189,NaN,3.03,14.98,0.0,0.0,3.20,0.0,None,1.0,19.18,NaN,NaN,NaN


In [7]:
df_train = read_data("./data/green_tripdata_2023-01.parquet")
df_val = read_data("./data/green_tripdata_2023-02.parquet")

In [8]:
len(df_train), len(df_val)

(65946, 62574)

In [9]:
df_train

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.90,...,0.5,4.03,0.0,None,1.0,24.18,1.0,1.0,2.75,11.016667
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.70,...,0.5,2.64,0.0,None,1.0,15.84,1.0,1.0,0.00,6.766667
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.20,...,0.5,1.94,0.0,None,1.0,11.64,1.0,1.0,0.00,6.333333
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.50,...,1.5,1.70,0.0,None,1.0,10.20,1.0,1.0,0.00,5.816667
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.00,...,1.5,0.00,0.0,None,1.0,8.00,1.0,1.0,0.00,5.966667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68206,2,2023-01-31 22:29:00,2023-01-31 22:42:00,None,NaN,49,62,NaN,4070.82,15.70,...,0.0,0.00,0.0,None,1.0,16.70,NaN,NaN,NaN,13.000000
68207,2,2023-01-31 22:40:00,2023-01-31 22:48:00,None,NaN,10,205,NaN,2.14,4.41,...,0.0,0.00,0.0,None,1.0,5.41,NaN,NaN,NaN,8.000000
68208,2,2023-01-31 23:46:00,2023-02-01 00:02:00,None,NaN,66,37,NaN,3.44,16.53,...,0.0,3.51,0.0,None,1.0,21.04,NaN,NaN,NaN,16.000000
68209,2,2023-01-31 23:01:00,2023-01-31 23:19:00,None,NaN,225,189,NaN,3.03,14.98,...,0.0,3.20,0.0,None,1.0,19.18,NaN,NaN,NaN,18.000000


In [10]:
categorical = ['PULocationID','DOLocationID']
df_onehot = pd.get_dummies(df_train,columns = categorical)

In [11]:
df_onehot.shape

(65946, 486)

In [12]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']

In [13]:
train_dicts = df_train[categorical+numerical].to_dict(orient = 'records')

In [14]:
train_dicts

[{'PULocationID': '166', 'DOLocationID': '143', 'trip_distance': 2.58},
 {'PULocationID': '24', 'DOLocationID': '43', 'trip_distance': 1.81},
 {'PULocationID': '223', 'DOLocationID': '179', 'trip_distance': 0.0},
 {'PULocationID': '41', 'DOLocationID': '238', 'trip_distance': 1.3},
 {'PULocationID': '41', 'DOLocationID': '74', 'trip_distance': 1.1},
 {'PULocationID': '41', 'DOLocationID': '262', 'trip_distance': 2.78},
 {'PULocationID': '181', 'DOLocationID': '45', 'trip_distance': 3.8},
 {'PULocationID': '24', 'DOLocationID': '75', 'trip_distance': 1.88},
 {'PULocationID': '41', 'DOLocationID': '166', 'trip_distance': 1.11},
 {'PULocationID': '24', 'DOLocationID': '140', 'trip_distance': 4.22},
 {'PULocationID': '255', 'DOLocationID': '234', 'trip_distance': 4.8},
 {'PULocationID': '75', 'DOLocationID': '140', 'trip_distance': 1.99},
 {'PULocationID': '75', 'DOLocationID': '148', 'trip_distance': 7.08},
 {'PULocationID': '66', 'DOLocationID': '255', 'trip_distance': 3.23},
 {'PULocati

In [15]:
dv = DictVectorizer()

train_dicts = df_train[categorical+numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical+numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [16]:
X_train.shape

(65946, 468)

In [17]:
X_val.shape

(62574, 468)

In [18]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [19]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [20]:
from sklearn.metrics import root_mean_squared_error

In [21]:
y_pred_train = lr.predict(X_train)
mean_squared_error(y_train, y_pred_train, squared=False)


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.060351540140314

In [22]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val,y_pred, squared=False)
root_mean_squared_error(y_val,y_pred)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.355377999391019

In [23]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [35]:
with mlflow.start_run():

    mlflow.set_tag("developer","anurag")

    mlflow.log_param("train-daat-path","./data/green_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path","./data/green_tripdata_2023-02.parquet")

    alpha =0.1
    mlflow.log_param("alpha",alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val,y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin",artifact_path="models_pickle")


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [26]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [29]:
train  = xgb.DMatrix(X_train,label = y_train)
valid = xgb.DMatrix(X_val,label =y_val)

In [30]:
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag("model","xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain = train,
            num_boost_round = 1000,
            evals = [(valid, "validation")],
            early_stopping_rounds = 50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val,y_pred, squared=False)
        mlflow.log_metric("rmse",rmse)

    return {'loss':rmse,'status':STATUS_OK}

In [31]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth',4,100,1)),
    'learning_rate':hp.loguniform('learning_rate',-3,0),
    'reg_alpha':hp.loguniform('reg_alpha',-5,-1),
    'reg_lambda':hp.loguniform('reg_lambda',-6,-1),
    'min_child_weight':hp.loguniform('min_child_weight',-1,3),
    'objective':'reg:linear',
    'seed':42,
}

best_result = fmin(
    fn = objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:5.43831                           
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:13:42] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:5.19132                           
[2]	validation-rmse:5.14021                           
[3]	validation-rmse:5.12467                           
[4]	validation-rmse:5.11982                           
[5]	validation-rmse:5.11663                           
[6]	validation-rmse:5.12658                           
[7]	validation-rmse:5.12301                           
[8]	validation-rmse:5.11767                           
[9]	validation-rmse:5.11912                           
[10]	validation-rmse:5.12682                          
[11]	validation-rmse:5.12521                          
[12]	validation-rmse:5.12529                          
[13]	validation-rmse:5.12505                          
[14]	validation-rmse:5.12591                          
[15]	validation-rmse:5.13065                          
[16]	validation-rmse:5.13105                          
[17]	validation-rmse:5.13163                          
[18]	validation-rmse:5.13051                          
[19]	valid

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:13:44] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.98673                                                    
[1]	validation-rmse:8.67462                                                    
[2]	validation-rmse:8.38478                                                    
[3]	validation-rmse:8.11555                                                    
[4]	validation-rmse:7.86583                                                    
[5]	validation-rmse:7.63431                                                    
[6]	validation-rmse:7.42089                                                    
[7]	validation-rmse:7.22386                                                    
[8]	validation-rmse:7.04211                                                    
[9]	validation-rmse:6.87447                                                    
[10]	validation-rmse:6.72080                                                   
[11]	validation-rmse:6.58008                                                   
[12]	validation-rmse:6.45045            

...

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:20:17] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.77220                                                     
[1]	validation-rmse:8.28524                                                     
[2]	validation-rmse:7.85592                                                     
[3]	validation-rmse:7.47881                                                     
[4]	validation-rmse:7.14908                                                     
[5]	validation-rmse:6.85993                                                     
[6]	validation-rmse:6.61076                                                     
[7]	validation-rmse:6.39456                                                     
[8]	validation-rmse:6.20777                                                     
[9]	validation-rmse:6.04733                                                     
[10]	validation-rmse:5.91073                                                    
[11]	validation-rmse:5.79411                                                    
[12]	validation-rmse:5.69466

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:20:31] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.68035                                                     
[1]	validation-rmse:8.12720                                                     
[2]	validation-rmse:7.65117                                                     
[3]	validation-rmse:7.24593                                                     
[4]	validation-rmse:6.89737                                                     
[5]	validation-rmse:6.60211                                                     
[6]	validation-rmse:6.35541                                                     
[7]	validation-rmse:6.14636                                                     
[8]	validation-rmse:5.97446                                                     
[9]	validation-rmse:5.83118                                                     
[10]	validation-rmse:5.71247                                                    
[11]	validation-rmse:5.61345                                                    
[12]	validation-rmse:5.53506

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [32]:
params = {
'learning_rate':0.08120115297016123,
'max_depth':7,
'min_child_weight':2.3550208026923514,
'objective':'reg:linear',
'reg_alpha':0.1281277112855484,
'reg_lambda':0.008079900465519313,
'seed':42
}

mlflow.xgboost.autolog()
booster = xgb.train(
    params=params,
    dtrain = train,
    num_boost_round = 1000,
    evals = [(valid, "validation")],
    early_stopping_rounds = 50
)



2024/05/26 12:46:04 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '4aad783f3a5c474e981964ebc8b3b243', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:8.84291
[1]	validation-rmse:8.41690
[2]	validation-rmse:8.03673
[3]	validation-rmse:7.69970
[4]	validation-rmse:7.40192
[5]	validation-rmse:7.13936
[6]	validation-rmse:6.91028
[7]	validation-rmse:6.70829
[8]	validation-rmse:6.53169
[9]	validation-rmse:6.37751
[10]	validation-rmse:6.24341
[11]	validation-rmse:6.12512
[12]	validation-rmse:6.02427
[13]	validation-rmse:5.93546
[14]	validation-rmse:5.85690


/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:46:04] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[15]	validation-rmse:5.79028
[16]	validation-rmse:5.73188
[17]	validation-rmse:5.67827
[18]	validation-rmse:5.63474
[19]	validation-rmse:5.59536
[20]	validation-rmse:5.56077
[21]	validation-rmse:5.52970
[22]	validation-rmse:5.50346
[23]	validation-rmse:5.48060
[24]	validation-rmse:5.46041
[25]	validation-rmse:5.44138
[26]	validation-rmse:5.42479
[27]	validation-rmse:5.40694
[28]	validation-rmse:5.39376
[29]	validation-rmse:5.38198
[30]	validation-rmse:5.36943
[31]	validation-rmse:5.36046
[32]	validation-rmse:5.35071
[33]	validation-rmse:5.34272
[34]	validation-rmse:5.33561
[35]	validation-rmse:5.32769
[36]	validation-rmse:5.32178
[37]	validation-rmse:5.31479
[38]	validation-rmse:5.30933
[39]	validation-rmse:5.30327
[40]	validation-rmse:5.29862
[41]	validation-rmse:5.29279
[42]	validation-rmse:5.28849
[43]	validation-rmse:5.28359
[44]	validation-rmse:5.27940
[45]	validation-rmse:5.27590
[46]	validation-rmse:5.27142
[47]	validation-rmse:5.26875
[48]	validation-rmse:5.26513
[49]	validatio

2024/05/26 12:46:17 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/26 12:46:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:46:17] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
